In [2]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [3]:
#CODE TO READ THE DATA

import pandas as pd

df_all = pd.read_json('/content/video_games_newer_all_data_with_rawg_final_for_train_v2.json')

df_all.head(2)


,Year_of_Release,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,User_Score,added,screenshots_count,achievements_count,reddit_count,twitch_count,youtube_count,ratings_count,suggestions_count,additions_count,game_series_count,reviews_count,platform_count
0,2002,9,0.49,0.38,0.26,0.13,1.27,8.5,35,18,0,0,0,0,12,547,0,8,12,1
1,2002,9,0.23,0.18,0.20,0.06,0.68,8.9,18,29,0,0,0,0,4,446,0,8,4,1


In [4]:
df_all.columns

Index(['Year_of_Release', 'Genre', 'NA_Sales', 'EU_Sales', 'JP_Sales',
       'Other_Sales', 'Global_Sales', 'User_Score', 'added',
       'screenshots_count', 'achievements_count', 'reddit_count',
       'twitch_count', 'youtube_count', 'ratings_count', 'suggestions_count',
       'additions_count', 'game_series_count', 'reviews_count',
       'platform_count'],
      dtype='object')

In [5]:
df_all = df_all[df_all.NA_Sales <= 6]
df_all = df_all.drop(columns = ['Year_of_Release', 'User_Score'])

In [6]:
cols = ['Genre']
df_all[cols] = df_all[cols].apply(lambda x: pd.factorize(x)[0] + 1)

In [7]:
df_all.head(2)

,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,added,screenshots_count,achievements_count,reddit_count,twitch_count,youtube_count,ratings_count,suggestions_count,additions_count,game_series_count,reviews_count,platform_count
0,1,0.49,0.38,0.26,0.13,1.27,35,18,0,0,0,0,12,547,0,8,12,1
1,1,0.23,0.18,0.20,0.06,0.68,18,29,0,0,0,0,4,446,0,8,4,1


In [8]:
#potential columns to drop

#df_all = df_all.drop(columns = ['Year_of_Release', 'screenshots_count', 'additions_count', 'game_series_count'])

In [9]:
df_all.isna().sum()

Genre                 0
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
added                 0
screenshots_count     0
achievements_count    0
reddit_count          0
twitch_count          0
youtube_count         0
ratings_count         0
suggestions_count     0
additions_count       0
game_series_count     0
reviews_count         0
platform_count        0
dtype: int64

In [10]:
#PREPARE FOR REGRESSION

training, test = train_test_split(df_all, test_size = 0.25, random_state = 32)
features = training.drop(['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'], axis = 1).columns

#pipeline.fit(training_features, training_labels)


In [11]:
scaler = StandardScaler()

In [12]:
training[['added', 'screenshots_count', 
     'achievements_count',	'reddit_count',	'twitch_count',	'youtube_count',
     'ratings_count',	'suggestions_count',	'additions_count',
     'game_series_count',	'reviews_count']] = scaler.fit_transform(training[['added', 'screenshots_count', 
     'achievements_count',	'reddit_count',	'twitch_count',	'youtube_count',
     'ratings_count',	'suggestions_count',	'additions_count',
     'game_series_count',	'reviews_count']])

In [13]:
test[['added', 'screenshots_count', 
     'achievements_count',	'reddit_count',	'twitch_count',	'youtube_count',
     'ratings_count',	'suggestions_count',	'additions_count',
     'game_series_count',	'reviews_count']] = scaler.transform(test[['added', 'screenshots_count', 
     'achievements_count',	'reddit_count',	'twitch_count',	'youtube_count',
     'ratings_count',	'suggestions_count',	'additions_count',
     'game_series_count',	'reviews_count']])

In [14]:

training_features, training_labels = training[features], training['NA_Sales']
test_features, test_labels = test[features], test['NA_Sales']

pipeline = Pipeline([('regressor', MLPRegressor())])

In [15]:
#MLP REGRESSION WITH HALVING GRID SEARCH


estimator=pipeline


param_grid = {'regressor__hidden_layer_sizes': [(50,50,50), (50,100,50), (100,1)],
          'regressor__activation': ['relu','tanh','logistic'],
          'regressor__alpha': [0.0001, 0.05],
          'regressor__learning_rate': ['constant','adaptive'],
          'regressor__solver': ['adam']}

gsc = HalvingGridSearchCV(
    estimator,
    param_grid,
    cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

grid_result = gsc.fit(training_features, training_labels)


best_params = grid_result.best_params_

print(best_params)

print(f'Best score: {grid_result.best_score_}')

# best_mlp = MLPRegressor(hidden_layer_sizes = best_params["hidden_layer_sizes"], 
#                         activation =best_params["activation"],
#                         solver=best_params["solver"],
#                         max_iter= 5000, n_iter_no_change = 200
#               )

# scoring = {
#            'abs_error': 'neg_mean_absolute_error',
#            'squared_error': 'neg_mean_squared_error',
#            'r2':'r2'}

# scores = cross_val_score(best_mlp, training_features, training_labels, cv=10, scoring=scoring)


n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 104
max_resources_: 2819
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 36
n_resources: 104
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 1
n_candidates: 12
n_resources: 312
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 4
n_resources: 936
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 3
n_candidates: 2
n_resources: 2808
Fitting 5 folds for each of 2 candidates, totalling 10 fits
{'regressor__activation': 'logistic', 'regressor__alpha': 0.0001, 'regressor__hidden_layer_sizes': (50, 50, 50), 'regressor__learning_rate': 'adaptive', 'regressor__solver': 'adam'}
Best score: -0.5060979505934061


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [16]:
#JAPAN SALES
training_features, training_labels = training[features], training['JP_Sales']
test_features, test_labels = test[features], test['JP_Sales']

pipeline = Pipeline([('regressor', MLPRegressor())])

#MLP REGRESSION WITH HALVING GRID SEARCH


estimator=pipeline


param_grid = {'regressor__hidden_layer_sizes': [(50,50,50), (50,100,50), (100,1)],
          'regressor__activation': ['relu','tanh','logistic'],
          'regressor__alpha': [0.0001, 0.05],
          'regressor__learning_rate': ['constant','adaptive'],
          'regressor__solver': ['adam']}

gsc = HalvingGridSearchCV(
    estimator,
    param_grid,
    cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

grid_result = gsc.fit(training_features, training_labels)


best_params = grid_result.best_params_

print(best_params)

print(f'Best score: {grid_result.best_score_}')

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 104
max_resources_: 2819
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 36
n_resources: 104
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 1
n_candidates: 12
n_resources: 312
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 4
n_resources: 936
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 3
n_candidates: 2
n_resources: 2808
Fitting 5 folds for each of 2 candidates, totalling 10 fits
{'regressor__activation': 'logistic', 'regressor__alpha': 0.05, 'regressor__hidden_layer_sizes': (50, 100, 50), 'regressor__learning_rate': 'constant', 'regressor__solver': 'adam'}
Best score: -0.07596431569428548


In [17]:
#EUROPE

training = training[training.EU_Sales < 2]
training_features, training_labels = training[features], training['EU_Sales']
test_features, test_labels = test[features], test['EU_Sales']

pipeline = Pipeline([('regressor', MLPRegressor())])

#MLP REGRESSION WITH HALVING GRID SEARCH


estimator=pipeline


param_grid = {'regressor__hidden_layer_sizes': [(50,50,50), (50,100,50), (100,1)],
          'regressor__activation': ['relu','tanh','logistic'],
          'regressor__alpha': [0.0001, 0.05],
          'regressor__learning_rate': ['constant','adaptive'],
          'regressor__solver': ['adam']}

gsc = HalvingGridSearchCV(
    estimator,
    param_grid,
    cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

grid_result = gsc.fit(training_features, training_labels)


best_params = grid_result.best_params_

print(best_params)

print(f'Best score: {grid_result.best_score_}')

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 101
max_resources_: 2745
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 36
n_resources: 101
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 1
n_candidates: 12
n_resources: 303
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 4
n_resources: 909
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 3
n_candidates: 2
n_resources: 2727
Fitting 5 folds for each of 2 candidates, totalling 10 fits
{'regressor__activation': 'logistic', 'regressor__alpha': 0.05, 'regressor__hidden_layer_sizes': (50, 100, 50), 'regressor__learning_rate': 'adaptive', 'regressor__solver': 'adam'}
Best score: -0.10917831219815186


In [18]:
#OTHER

training = training[training.Other_Sales < 2]
training_features, training_labels = training[features], training['Other_Sales']
test_features, test_labels = test[features], test['Other_Sales']

pipeline = Pipeline([('regressor', MLPRegressor())])

#MLP REGRESSION WITH HALVING GRID SEARCH


estimator=pipeline


param_grid = {'regressor__hidden_layer_sizes': [(50,50,50), (50,100,50), (100,1)],
          'regressor__activation': ['relu','tanh','logistic'],
          'regressor__alpha': [0.0001, 0.05],
          'regressor__learning_rate': ['constant','adaptive'],
          'regressor__solver': ['adam']}

gsc = HalvingGridSearchCV(
    estimator,
    param_grid,
    cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

grid_result = gsc.fit(training_features, training_labels)


best_params = grid_result.best_params_

print(best_params)

print(f'Best score: {grid_result.best_score_}')

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 101
max_resources_: 2743
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 36
n_resources: 101
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 1
n_candidates: 12
n_resources: 303
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 4
n_resources: 909
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 3
n_candidates: 2
n_resources: 2727
Fitting 5 folds for each of 2 candidates, totalling 10 fits
{'regressor__activation': 'logistic', 'regressor__alpha': 0.0001, 'regressor__hidden_layer_sizes': (50, 100, 50), 'regressor__learning_rate': 'constant', 'regressor__solver': 'adam'}
Best score: -0.01632916398738664


In [19]:
df_all = pd.read_json('/content/video_games_newer_all_data_with_rawg_final_for_train_v2.json')

In [20]:
df_all = df_all[df_all.NA_Sales <= 6]
df_all = df_all[df_all.EU_Sales <= 2]
df_all = df_all[df_all.JP_Sales <= 0.7]
df_all = df_all[df_all.Other_Sales <= 2]
df_all = df_all[df_all.Global_Sales <= 6]
df_all = df_all.drop(columns = ['Year_of_Release','Global_Sales'])

In [21]:
len(df_all)

3553

In [22]:
#PREPARE DATA FOR CLASSIFICATION
import numpy as np

tmp = df_all.copy()
tmp['User_Score'] = np.where(tmp['User_Score'] < 7.5, 0, tmp['User_Score'])
tmp['User_Score'] = np.where(tmp['User_Score'] >= 7.5, 1, tmp['User_Score'])
tmp['User_Score'] = tmp['User_Score'].astype('category').cat.codes
# tmp['User_Score'] = np.where(tmp['User_Score'] < 6.0, 0, tmp['User_Score'])
# tmp['User_Score'] = np.where((6.0 <= tmp['User_Score']) & ( tmp['User_Score'] < 7.0), 1 ,tmp['User_Score'])
# tmp['User_Score'] = np.where((7.0 <= tmp['User_Score']) & ( tmp['User_Score'] < 7.5), 2 ,tmp['User_Score'])
# tmp['User_Score'] = np.where((7.5 <= tmp['User_Score']) & ( tmp['User_Score'] < 8.0), 3 ,tmp['User_Score'])
# tmp['User_Score'] = np.where((8.0 <= tmp['User_Score']) & ( tmp['User_Score'] < 8.5), 4 ,tmp['User_Score'])
# tmp['User_Score'] = np.where(8.5 <= tmp['User_Score'] , 5 ,tmp['User_Score'])
# tmp['User_Score'] = tmp['User_Score'].astype('category').cat.codes


In [23]:
training, test = train_test_split(tmp, test_size = 0.25, random_state = 32)
features = training.drop(['User_Score'], axis = 1).columns

In [24]:
scaler = StandardScaler()

In [25]:
training[['added', 'screenshots_count', 
     'achievements_count',	'reddit_count',	'twitch_count',	'youtube_count',
     'ratings_count',	'suggestions_count',	'additions_count',
     'game_series_count',	'reviews_count','NA_Sales','EU_Sales','JP_Sales','Other_Sales']] = scaler.fit_transform(training[['added', 'screenshots_count', 
     'achievements_count',	'reddit_count',	'twitch_count',	'youtube_count',
     'ratings_count',	'suggestions_count',	'additions_count',
     'game_series_count',	'reviews_count','NA_Sales','EU_Sales','JP_Sales','Other_Sales']])

In [26]:
test[['added', 'screenshots_count', 
     'achievements_count',	'reddit_count',	'twitch_count',	'youtube_count',
     'ratings_count',	'suggestions_count',	'additions_count',
     'game_series_count',	'reviews_count','NA_Sales','EU_Sales','JP_Sales','Other_Sales']] = scaler.transform(test[['added', 'screenshots_count', 
     'achievements_count',	'reddit_count',	'twitch_count',	'youtube_count',
     'ratings_count',	'suggestions_count',	'additions_count',
     'game_series_count',	'reviews_count','NA_Sales','EU_Sales','JP_Sales','Other_Sales']])

In [27]:

training_features, training_labels = training[features], training['User_Score']
test_features, test_labels = test[features], test['User_Score']


pipeline = Pipeline([('classifier', MLPClassifier(max_iter=100))])

In [28]:
#MLP CLASSIFICATION WITH HALVING GRID SEARCH

mlp_gs = pipeline
parameter_space = {
    'classifier__hidden_layer_sizes': [(20,50,20),(10,30,10),(20,),(100, 50),(100,50,20)],
    'classifier__activation': [ 'tanh', 'relu'],
    'classifier__solver': ['adam', 'sgd'],
    'classifier__alpha': [0.0001, 0.001, 0.05, 0.1],
    'classifier__learning_rate': ['adaptive', 'constant'],
    'classifier__batch_size' : [16,32,64,128]
}


clf = HalvingGridSearchCV(mlp_gs, parameter_space, verbose=1, n_jobs=-1, cv=5, scoring='f1_macro')

grid_result = clf.fit(training_features, training_labels)


best_params = grid_result.best_params_

print(best_params)

print(f'Best score: {grid_result.best_score_}')

n_iterations: 5
n_required_iterations: 6
n_possible_iterations: 5
min_resources_: 20
max_resources_: 2664
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 640
n_resources: 20
Fitting 5 folds for each of 640 candidates, totalling 3200 fits
----------
iter: 1
n_candidates: 214
n_resources: 60
Fitting 5 folds for each of 214 candidates, totalling 1070 fits
----------
iter: 2
n_candidates: 72
n_resources: 180
Fitting 5 folds for each of 72 candidates, totalling 360 fits
----------
iter: 3
n_candidates: 24
n_resources: 540
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 4
n_candidates: 8
n_resources: 1620
Fitting 5 folds for each of 8 candidates, totalling 40 fits
{'classifier__activation': 'tanh', 'classifier__alpha': 0.05, 'classifier__batch_size': 32, 'classifier__hidden_layer_sizes': (100, 50, 20), 'classifier__learning_rate': 'adaptive', 'classifier__solver': 'adam'}
Best score: 0.6018835594664721


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [32]:
#SVM CLASSIFICATION WITH HALVING GRID SEARCH
from sklearn.svm import SVC

pipeline = Pipeline([('SVM', SVC())])
parameteres = {'SVM__C':[0.001,0.1,10,100,1000,10e5], 
               'SVM__gamma':[0.1,0.01,0.001,0.0001],
               'SVM__kernel': ['rbf']}


clf = HalvingGridSearchCV(pipeline, 
                          parameteres, 
                          cv=5, 
                          factor=3, 
                          min_resources='exhaust', 
                          verbose=1, 
                          scoring='f1_macro')

clf.fit(training_features, training_labels)


print("Best parameters:", clf.best_params_)
print("Best f1-macro: ", clf.best_score_)


n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 296
max_resources_: 2664
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 24
n_resources: 296
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 1
n_candidates: 8
n_resources: 888
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 2
n_candidates: 3
n_resources: 2664
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters: {'SVM__C': 100, 'SVM__gamma': 0.01, 'SVM__kernel': 'rbf'}
Best f1-macro:  0.620278099390217


In [35]:
import pandas as pd
df_steam = pd.read_json('/content/steamspy_all.json')

df_steam.head(2)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,570,Dota 2,Valve,Valve,,1596223,333113,0,"200,000,000 .. 500,000,000",38179,1530,904,754,0.0,0.0,0.0,470308
1,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,6129357,805515,0,"50,000,000 .. 100,000,000",29163,864,6238,346,0.0,0.0,0.0,1035784


In [36]:
df_steam.isna().sum()

appid               0
name                0
developer           0
publisher           0
score_rank          0
positive            0
negative            0
userscore           0
owners              0
average_forever     0
average_2weeks      0
median_forever      0
median_2weeks       0
price              29
initialprice       22
discount           22
ccu                 0
dtype: int64

In [37]:
df_steam = df_steam.dropna()

In [38]:
df_steam.isna().sum()

appid              0
name               0
developer          0
publisher          0
score_rank         0
positive           0
negative           0
userscore          0
owners             0
average_forever    0
average_2weeks     0
median_forever     0
median_2weeks      0
price              0
initialprice       0
discount           0
ccu                0
dtype: int64

In [39]:
len(df_steam)

60782

In [40]:
df_steam['userscore'].unique()

array([  0,  95,  69,  55,  46,  73,  77,  97,  81,  74,  98,  67,  84,
        59,  87,  57,  65,  80,  51,  63,  68,  78,  82,  94, 100,  96,
        60,  61,  53,  76,  92,  71,  88])

In [41]:
df_steam['owners'][:2]

0    200,000,000 .. 500,000,000
1     50,000,000 .. 100,000,000
Name: owners, dtype: object

In [43]:
df_rawg = pd.read_json('/content/rawg_games_details_data_1.json')

df_rawg.head(2)

,id,slug,name,name_original,description,metacritic,metacritic_platforms,released,tba,updated,...,platforms,stores,developers,genres,tags,publishers,esrb_rating,clip,description_raw,community_rating
0,25057,wii-sports,Wii Sports,Wii Sports,<p>This is what video games should be: fun for...,76.0,"[{'metascore': 76, 'url': 'https://www.metacri...",2006-11-19,False,2023-02-19T20:34:26,...,"[{'platform': {'id': 11, 'name': 'Wii', 'slug'...","[{'id': 29436, 'url': '', 'store': {'id': 6, '...",[],"[{'id': 15, 'name': 'Sports', 'slug': 'sports'...","[{'id': 37796, 'name': 'exclusive', 'slug': 'e...","[{'id': 10681, 'name': 'Nintendo', 'slug': 'ni...",None,NaN,This is what video games should be: fun for ev...,NaN
1,25080,super-mario-bros,Super Mario Bros.,Super Mario Bros.,<p>Mario and Luigi star in their first ever Mu...,NaN,[],1985-09-13,False,2023-02-24T18:30:20,...,"[{'platform': {'id': 24, 'name': 'Game Boy Adv...","[{'id': 29465, 'url': '', 'store': {'id': 6, '...","[{'id': 16257, 'name': 'Nintendo', 'slug': 'ni...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","[{'id': 10681, 'name': 'Nintendo', 'slug': 'ni...",None,NaN,Mario and Luigi star in their first ever Mushr...,NaN


In [44]:
df_rawg['name_original'].nunique()

5870

In [45]:
result = pd.merge(df_rawg, df_steam, how="inner", left_on = "name_original", right_on = "name")

In [46]:
len(result)

641